# Remakes 

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
import requests
import re

## Acquiring the data from Wikipedia

We've acquired the data from this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_film_remakes_(A%E2%80%93M)). Since the data are in the tables, we extract the table elements using the `BeautifulSoup` library.

In [3]:
urls = [
    "https://en.wikipedia.org/wiki/List_of_film_remakes_(A%E2%80%93M)#A",
    "https://en.wikipedia.org/wiki/List_of_film_remakes_(N%E2%80%93Z)#Z",
]
tables = []

for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find_all("table", class_="wikitable")
    tables.extend(table)

## Parsing the Webpage Tables

After acquiring the data, we use the `BeatingSoup` library to parse the tables into a structure dataframe format. Then, we will do a manual post-clean up to ensure the data is in the correct format.


In [4]:
def unwrap(cell):
    """
    Unwrap a cell from its <p> tag if it exists.
    """
    if cell.p:
        return cell.p
    return cell


def parse_cell(cell):
    """
    Parse a cell with a single-entry in it.
    The expected format is "Title (YYYY)" or "Title (YYYY-YYYY)", or even "Title (YYYY-YY)".
    Also, we expect a hyperlink to the wikipedia page of the movie.

    Args:
        cell: BeautifulSoup tag of the cell.

    Returns:
        title: str, title of the movie.
        year: str, year of the movie. `None` if not found.
        url: str, wikipedia page of the movie. `None` if not found.
    """
    url = cell.a["href"].strip() if cell.a else None
    title_year = cell.get_text()
    title = re.sub(r"\((\d{4}(?:-\d{2,4})?)\)", "", title_year).strip()
    match = re.search(r"\((\d{4}(?:-\d{2,4})?)\)", title_year)
    if match:
        year = match.group(1)
    else:
        year = None
    return title, year, url


def parse_multi_cell(cell):
    """
    Parse a cell with multiple entries in it.
    The expected format is a list of <p> tags, <i> tags, and <a> tags.
    
    Args:
        cell: BeautifulSoup tag of the cell.
        
    Returns:
        titles: list, titles of the movies.
        years: list, years of the movies. `None` for each entry if not found.
        urls: list, wikipedia page of the movies. `None` for each if not found.
    """
    titles = []
    years = []
    urls = []
    years = re.findall(r"\((\d{4}(?:-\d{2,4})?)\)", cell.get_text())
    for tag in cell.find_all(["p", "i", "a"], recursive=False):
        tag = unwrap(tag)
        url = tag.a["href"].strip() if tag.a else None
        title = tag.get_text()
        titles.append(title)
        urls.append(url)
    if len(title) != len(years) and len(years) == 0:
        # NOTE: fallback to filling all years with None if no years has been found.
        years = [None] * len(titles)
    assert len(titles) == len(years) == len(urls)
    return titles, years, urls


def convert_to_df(table):
    rows = table.find_all("tr")
    data = {
        "original": [],
        "original_year": [],
        "original_url": [],
        "remake": [],
        "remake_year": [],
        "remake_url": [],
    }
    for row in rows[1:]:
        cols = row.find_all(["td", "th"])
        if len(cols) == 0:
            continue
        if len(cols) < 3:
            remake_titles, remake_years, remake_urls = parse_multi_cell(cols[0])
            original_title = data["original"][-1]
            original_year = data["original_year"][-1]
            original_url = data["original_url"][-1]
        else:
            original_title, original_year, original_url = parse_cell(cols[0])
            remake_titles, remake_years, remake_urls = parse_multi_cell(cols[1])
        for remake_title, remake_year, remake_url in zip(
            remake_titles, remake_years, remake_urls
        ):
            data["original"].append(original_title)
            data["original_year"].append(original_year)
            data["original_url"].append(original_url if original_url else None)
            data["remake"].append(remake_title)
            data["remake_year"].append(remake_year)
            data["remake_url"].append(remake_url if remake_url else None)

    df = pd.DataFrame(data)
    return df

In [5]:
df_list = []
for table in tables:
    df = convert_to_df(table)
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)
df

,original,original_year,original_url,remake,remake_year,remake_url
0,12 Angry Men,1957,/wiki/12_Angry_Men_(1957_film),12 Angry Men (1997),1997,None
1,13 Assassins,1963,/wiki/13_Assassins_(1963_film),13 Assassins,2010,/wiki/13_Assassins_(2010_film)
2,13 Ghosts,1960,/wiki/13_Ghosts,Thirteen Ghosts,2001,/wiki/Thirteen_Ghosts
3,13 Tzameti,2005,/wiki/13_Tzameti,13,2010,/wiki/13_(2010_film)
4,3 Idiots,2009,/wiki/3_Idiots,Nanban,2012,/wiki/Nanban_(2012_film)
...,...,...,...,...,...,...
733,Yellow Sky,1948,/wiki/Yellow_Sky,The Jackals,1967,/wiki/The_Jackals
734,Yojimbo,1961,/wiki/Yojimbo_(film),A Fistful of Dollars,1964,/wiki/A_Fistful_of_Dollars
735,Yojimbo,1961,/wiki/Yojimbo_(film),Last Man Standing,1996,/wiki/Last_Man_Standing_(1996_film)
736,"Yours, Mine and Ours",1968,"/wiki/Yours,_Mine_and_Ours_(1968_film)","Yours, Mine & Ours",2005,"/wiki/Yours,_Mine_%26_Ours_(2005_film)"


## Post Cleaning

In [6]:
df[df["original_year"].isnull() | df["remake_year"].isnull()]

,original,original_year,original_url,remake,remake_year,remake_url
246,The Godfather (1972–90),None,/wiki/The_Godfather_(film_series),Godfather (2007),2007,/wiki/Godfather_(2007_film)
347,Kaththi,2014,/wiki/Kaththi,Khaidhi No. 150,None,None
452,My Favorite Wife,1940,/wiki/My_Favorite_Wife,Something's Got to Give,None,/wiki/Something%27s_Got_to_Give


### Manual Intervention

In [7]:
df.loc[246, ['original', 'original_year', 'remake', 'remake_year']] = ["The Godfather", "1972", "Godfather", "2007"]
df.iloc[246]

original                             The Godfather
original_year                                 1972
original_url     /wiki/The_Godfather_(film_series)
remake                                   Godfather
remake_year                                   2007
remake_url             /wiki/Godfather_(2007_film)
Name: 246, dtype: object

In [8]:
df.loc[347, ['remake_year']] = ["2017"]
df.iloc[347]

original                 Kaththi
original_year               2014
original_url       /wiki/Kaththi
remake           Khaidhi No. 150
remake_year                 2017
remake_url                  None
Name: 347, dtype: object

In [9]:
df.loc[452, ['remake_year']] = ["1962"]
df.iloc[452]

original                        My Favorite Wife
original_year                               1940
original_url              /wiki/My_Favorite_Wife
remake                   Something's Got to Give
remake_year                                 1962
remake_url       /wiki/Something%27s_Got_to_Give
Name: 452, dtype: object

### Formatting

In [10]:
df["original_year"] = df["original_year"].astype(int)
df["remake_year"] = df["remake_year"].astype(int)
df

,original,original_year,original_url,remake,remake_year,remake_url
0,12 Angry Men,1957,/wiki/12_Angry_Men_(1957_film),12 Angry Men (1997),1997,None
1,13 Assassins,1963,/wiki/13_Assassins_(1963_film),13 Assassins,2010,/wiki/13_Assassins_(2010_film)
2,13 Ghosts,1960,/wiki/13_Ghosts,Thirteen Ghosts,2001,/wiki/Thirteen_Ghosts
3,13 Tzameti,2005,/wiki/13_Tzameti,13,2010,/wiki/13_(2010_film)
4,3 Idiots,2009,/wiki/3_Idiots,Nanban,2012,/wiki/Nanban_(2012_film)
...,...,...,...,...,...,...
733,Yellow Sky,1948,/wiki/Yellow_Sky,The Jackals,1967,/wiki/The_Jackals
734,Yojimbo,1961,/wiki/Yojimbo_(film),A Fistful of Dollars,1964,/wiki/A_Fistful_of_Dollars
735,Yojimbo,1961,/wiki/Yojimbo_(film),Last Man Standing,1996,/wiki/Last_Man_Standing_(1996_film)
736,"Yours, Mine and Ours",1968,"/wiki/Yours,_Mine_and_Ours_(1968_film)","Yours, Mine & Ours",2005,"/wiki/Yours,_Mine_%26_Ours_(2005_film)"


## Save the Output

In [11]:
root_dir = Path("../data/remake")
root_dir.mkdir(parents=True, exist_ok=True)
df.to_csv(root_dir / "remakes.csv", index=False)